<a href="https://colab.research.google.com/github/harsha-bsm/LDA-on-REDMI-NOTE8-reviews-in-FLIPKART/blob/master/Preprocessing_and_Modelling_LDA_SAMSUNGM30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import numpy as np
import pandas as pd

In [ ]:
data=pd.read_csv("samsungdata.csv")

In [ ]:
data.head()

,Unnamed: 0,Model,rating,user,time_of_review,review,title,user_address,user_type
0,0,Samsung Galaxy M30s Blue 128 Gb Reviews: Lates...,5,Aryan Patil,2months ago,Bokeh mode is very excellent than any other ha...,Wonderful,Latur District,Certified Buyer
1,1,Samsung Galaxy M30s Blue 128 Gb Reviews: Lates...,5,Flipkart Customer,5months ago,Just awsome product the picture quality is goo...,Great product,North Twenty Four Parganas District,Certified Buyer
2,2,Samsung Galaxy M30s Blue 128 Gb Reviews: Lates...,4,Suresh Kumar,5months ago,Samsung galaxy m30s is most useful.But Samsung...,Nice product,New Delhi,Certified Buyer
3,3,Samsung Galaxy M30s Blue 128 Gb Reviews: Lates...,1,Rohit Purba,6months ago,Not worth for me. i buy for my samsung tv this...,Utterly Disappointed,Rajpura,Certified Buyer
4,4,Samsung Galaxy M30s Blue 128 Gb Reviews: Lates...,5,Flipkart Customer,5months ago,"BSOLUTELY BEST PHONE, GO FOR THIS PHONELET ME ...",Highly recommended,Muzaffarpur,Certified Buyer


In [ ]:
pip install emot 

In [ ]:
####### Apply emoticons and emojis

from emot.emo_unicode import UNICODE_EMO, EMOTICONS # contains english text of emojis and emoticons in dict format 
def convert_emojis(review):
  for x in review:
    if x in UNICODE_EMO.keys():
      review=review.replace(x,"")
  return review

In [ ]:
#converting all basic contracted foms  
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
#removing special characters and tabs ,spaces,new lines etc..
def removecharacters(review):
  review= re.sub('[^A-Za-z0-9]+',' ',review)  #anything exept numbers and alphabets, replace them with space
  review=re.sub(r"\n"," ",review)  #new lines into space
  review=re.sub(r"\t"," ",review)  #tabs into space
  review=re.sub(r"\v"," ",review)  #vertical tab into space
  review=re.sub(r"\s"," ",review)   #all extra spaces into single space
  return review  


In [ ]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step
# ignore words like won't, haven't etc.. We have already taken care of them

stopword= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't","not","phone","product","mobile"])

In [ ]:
from nltk.corpus import stopwords
import nltk

In [ ]:
def removestopwords(review):
  return " ".join([word for word in review.split(" ") if not word in stopword])


In [ ]:
data["review"]=data["review"].apply(lambda x:convert_emojis(x))
data["review"]=data["review"].apply(lambda x:x.lower()) 
data["review"]=data["review"].apply(lambda x:decontracted(x))
data["review"]=data["review"].apply(lambda x:removecharacters(x))
data["review"]=pd.DataFrame(data["review"].apply(lambda x:removestopwords(x)))

In [ ]:
data["review"]

0      bokeh mode excellent handset budget battery ca...
1      awsome picture quality good battery backup aws...
2      samsung galaxy m30s useful samsung improve sel...
3      worth buy samsung tv live cast smartthins app ...
4      bsolutely best go phonelet clear features1 cam...
                             ...                        
293                                                 okay
294                           super style good look nice
295                                                 good
296                                                 good
297                                                 good
Name: review, Length: 298, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import ldamodel
import gensim.corpora
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
#https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-plot-topics-extraction-with-nmf-lda-py
def print_top_words(model, feature_names, n_top_words):
  topics_cluster=[]
  for topic_idx, topic in enumerate(model.components_):
      message = "Topic #%d: " % topic_idx
      message += ",".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
      topics_cluster.append([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
  return topics_cluster
  

### BOW and LDA 

In [ ]:
bow=CountVectorizer(ngram_range=(1,2))
bow_review=bow.fit_transform(data["review"])
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=15,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),5)

[['wesome', 'bad', 'thanks', 'xcellent', 'love'],
 ['good', 'battery', 'xcellent', 'excellent', 'display'],
 ['no', 'best', 'use', 'hangs', 'nice'],
 ['good', 'poor', 'camera good', 'camera', 'hand'],
 ['camera', 'money', 'awesome', 'wesome', 'worth money'],
 ['samsung', 'buy', 'best', 'buy online', 'online'],
 ['good', 'samsung', 'nice', 'best', 'loved'],
 ['good', 'battery', 'battery backup', 'backup', 'expected'],
 ['good', 'quality', 'camera', 'price', 'battery'],
 ['nice', 'good', 'battery', 'camera', 'thanks'],
 ['superb', 'hours', 'display', 'best', 'battery'],
 ['super', 'battery', 'battery life', 'life', 'best'],
 ['great', 'flipkart', 'worst', 'phones', 'camera'],
 ['switch', 'camera quality', 'per', 'quality', 'though'],
 ['best', 'good', 'battery', 'camera', 'awsome']]

#### LDA on topics obtained

In [ ]:
arr=np.array(print_top_words(lda,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,2))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=4,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),4)


[['good', 'camera', 'backup', 'display'],
 ['best', 'money', 'quality', 'worth'],
 ['nice', 'xcellent', 'great', 'loved'],
 ['battery', 'online', 'buy', 'samsung']]

### TFIDF AND LDA

In [ ]:
tfidf=TfidfVectorizer(ngram_range=(1,2))
tfidf_words=tfidf.fit_transform(data["review"])
words=tfidf.get_feature_names()
df_bow=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=10,random_state=0)
lda.fit_transform(tfidf_words)
print_top_words(lda,words,8)

[['price',
  'suprrrrrrrrr',
  'nice',
  'battery',
  'good price',
  'xcellent battery',
  'nice phones',
  'nice set'],
 ['xcellent',
  'excellent',
  'best',
  'display',
  'thakarthu',
  'waysome',
  'fresh',
  'wsm'],
 ['fantastic',
  'nicc',
  'battery',
  'nice',
  'camera',
  'xcellent smartphone',
  'good',
  'nice camera'],
 ['superb',
  'good',
  'smartphone',
  'sm',
  'battery',
  'good choic',
  'choic',
  'no'],
 ['camera',
  'super',
  'worst',
  'normal',
  'rattan',
  'utstanding',
  'good',
  'quality'],
 ['super',
  'best',
  'bad',
  'flipkart',
  'hangs',
  'frequently',
  'quality battery',
  'worth'],
 ['good',
  'one',
  'samsung',
  'good samsung',
  'good one',
  'great',
  'lovely',
  'top'],
 ['wesome',
  'ok',
  'value money',
  'value',
  'money',
  'working',
  'camera',
  'isplay good'],
 ['exellent',
  'superrrrbbbbb',
  'good',
  'price',
  'super nice',
  'nice',
  'nice cell',
  'cell'],
 ['nice', 'awesome', 'no', 'battery', 'good', 'thanks', 'loved

##### LDA on topics obtained 

In [ ]:
arr=np.array(print_top_words(lda,words,5)).reshape(1,-1)[0]
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(arr)
words=tfidf.get_feature_names()
df_tfidf=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=4,random_state=0)
lda.fit_transform(df_tfidf)
print_top_words(lda,words,4)


[['good', 'battery', 'nice', 'nicc'],
 ['super', 'camera', 'one', 'xcellent'],
 ['best', 'worst', 'smartphone', 'flipkart'],
 ['price', 'money', 'value', 'samsung']]

### NMF - TFIDF

In [ ]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [ ]:
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(data["review"])
words=tfidf.get_feature_names()
df_lda=pd.DataFrame(tfidf_words.toarray(),columns=words)


In [ ]:
nmf = NMF(n_components=10, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)

print_top_words(nmf,words,6)

[['good', 'one', 'normal', 'choic', 'isplay', 'choice'],
 ['nice', 'awsm', 'phones', 'samsang', 'cell', 'osm'],
 ['super', 'quality', 'camera', 'also', 'performance', 'look'],
 ['wesome', 'outstanding', 'perfomance', 'loving', 'slow', 'gaming'],
 ['best', 'samsung', 'price', 'loved', 'range', 'buy'],
 ['battery', 'display', 'camera', 'backup', 'no', 'poor'],
 ['superb', 'expected', 'great', 'utstanding', 'got', 'galaxy'],
 ['xcellent', 'flipkart', 'smartphone', 'love', 'excellent', 'thanks'],
 ['money', 'awesome', 'value', 'much', 'delivery', 'worth'],
 ['bad', 'ok', 'thank', 'worst', 'hangs', 'working']]

#### NMF on topics obtained

In [ ]:
arr=np.array(print_top_words(nmf,words,5)).reshape(1,-1)[0]

In [ ]:
tfidf=TfidfVectorizer(ngram_range=(1,2))
tfidf_words=tfidf.fit_transform(arr)
words=tfidf.get_feature_names()
df_lda=pd.DataFrame(tfidf_words.toarray(),columns=words)
nmf = NMF(n_components=5, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)

print_top_words(nmf,words,4)

[['camera', 'quality', 'one', 'performance'],
 ['much', 'price', 'hangs', 'good'],
 ['also', 'slow', 'nice', 'normal'],
 ['xcellent', 'choic', 'wesome', 'awesome'],
 ['love', 'worst', 'great', 'cell']]

### NMF - BOW

In [ ]:
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(data["review"])
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
nmf = NMF(n_components=10, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
bow_words=bow.get_feature_names()
print_top_words(nmf,bow_words,5)

[['good', 'battery', 'camera', 'display', 'performance'],
 ['super', 'superb', 'great', 'display', 'hours'],
 ['samsung', 'buy', 'm30s', 'loved', 'online'],
 ['nice', 'delivery', 'thanks', 'awsm', 'look'],
 ['quality', 'camera', 'hangs', 'using', 'fine'],
 ['best', 'price', 'range', 'camera', 'battery'],
 ['wesome', 'battery', 'money', 'xcellent', 'bad'],
 ['performance', 'excellent', 'no', 'display', 'budget'],
 ['use', 'ok', 'best', 'awsome', 'backup'],
 ['flipkart', 'thank', 'love', 'worst', 'better']]

#### NMF on topics obtained ------------------------------------------

In [ ]:
arr=np.array(print_top_words(nmf,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
nmf = NMF(n_components=4, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
bow_words=bow.get_feature_names()
print_top_words(nmf,bow_words,4)

[['camera', 'awsome', 'using', 'thanks'],
 ['battery', 'good', 'use', 'm30s'],
 ['display', 'xcellent', 'ok', 'hours'],
 ['performance', 'best', 'quality', 'hangs']]

## POS TAGGING taking only nouns

In [ ]:
  import nltk
  nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
def only_nouns(review):
  lis= " ".join([each.strip(" ") for each,pos in nltk.pos_tag(review.split()) if pos=="NN"])
  return lis
  


In [ ]:
data["review"]=data["review"].apply(lambda x:only_nouns(x))

### LDA BOW

In [ ]:
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(data["review"])
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=10,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),4)

[['price', 'battery', 'quality', 'range'],
 ['battery', 'camera', 'backup', 'performance'],
 ['superb', 'samsung', 'flipkart', 'delivery'],
 ['battery', 'wesome', 'display', 'camera'],
 ['super', 'use', 'xcellent', 'performance'],
 ['online', 'buy', 'call', 'function'],
 ['nice', 'box', 'thank', 'excellent'],
 ['money', 'value', 'delivery', 'osm'],
 ['display', 'battery', 'camera', 'problem'],
 ['camera', 'quality', 'performance', 'battery']]

#### (LDA again - on topics obtained) 

In [ ]:
arr=np.array(print_top_words(lda,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,2))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=4,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),4)


[['excellent', 'thank', 'performance', 'super'],
 ['box', 'samsung', 'bit', 'online'],
 ['camera', 'battery', 'quality', 'osm'],
 ['value', 'problem', 'money', 'team']]

### LDA TFIDF

In [ ]:
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(data["review"])
words=tfidf.get_feature_names()
df_bow=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=8,random_state=0)
lda.fit_transform(tfidf_words)
print_top_words(lda,words,4)

[['awesome', 'excellent', 'isplay', 'sm'],
 ['camera', 'super', 'quality', 'm30s'],
 ['superb', 'samsung', 'delivery', 'love'],
 ['battery', 'price', 'wesome', 'backup'],
 ['ok', 'smartphone', 'use', 'purchase'],
 ['money', 'value', 'call', 'delivery'],
 ['nice', 'experience', 'function', 'thank'],
 ['ear', 'perfomance', 'android', 'butter']]

#### (LDA again - on topics obtained) 

In [ ]:
arr=np.array(print_top_words(lda,words,5)).reshape(1,-1)[0]
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(arr)
words=tfidf.get_feature_names()
df_tfidf=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=4,random_state=0)
lda.fit_transform(df_tfidf)
print_top_words(lda,words,4)


[['super', 'ear', 'glass', 'butter'],
 ['delivery', 'm30s', 'use', 'rattan'],
 ['camera', 'display', 'isplay', 'wesome'],
 ['experience', 'ok', 'thank', 'mi']]

### NMF BOW (better results)

In [ ]:
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(data["review"])


In [ ]:
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(data["review"])

nmf = NMF(n_components=10, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
bow_words=bow.get_feature_names()
print_top_words(nmf,bow_words,5)

[['quality', 'value', 'wesome', 'mah', 'backup'],
 ['money', 'flipkart', 'value', 'backup', 'battery'],
 ['samsung', 'purchase', 'wesome', 'mah', 'backup'],
 ['superb', 'display', 'wesome', 'mah', 'backup'],
 ['time', 'love', 'wesome', 'mah', 'backup'],
 ['awesome', 'value', 'backup', 'battery', 'budget'],
 ['battery', 'mah', 'wesome', 'value', 'backup'],
 ['budget', 'performance', 'wesome', 'mah', 'backup'],
 ['backup', 'super', 'camera', 'wesome', 'mah'],
 ['wesome', 'nice', 'mah', 'backup', 'battery']]

#### (NMF again - on topics obtained) 

In [ ]:
arr=np.array(print_top_words(nmf,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
nmf = NMF(n_components=5, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
bow_words=bow.get_feature_names()
print_top_words(nmf,bow_words,4)

[['battery', 'wesome', 'value', 'time'],
 ['value', 'backup', 'wesome', 'time'],
 ['time', 'super', 'samsung', 'quality'],
 ['wesome', 'value', 'time', 'superb'],
 ['mah', 'superb', 'love', 'wesome']]

### NMF TFIDF

In [ ]:
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(data["review"])
words=tfidf.get_feature_names()

In [ ]:
nmf = NMF(n_components=10, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)
print_top_words(nmf,words,4)

[['battery', 'backup', 'camera', 'performance'],
 ['camera', 'quality', 'experience', 'light'],
 ['wesome', 'life', 'model', 'battery'],
 ['price', 'range', 'segment', 'mid'],
 ['super', 'flipkart', 'purchase', 'use'],
 ['superb', 'delivery', 'condition', 'galaxy'],
 ['money', 'value', 'time', 'delivery'],
 ['samsung', 'love', 'm30s', 'performance'],
 ['display', 'ok', 'xcellent', 'smartphone'],
 ['nice', 'awesome', 'osm', 'mi']]

#### (NMF again - on topics obtained) ---------------------------------------------------

In [ ]:
arr=np.array(print_top_words(nmf,words,5)).reshape(1,-1)[0]
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(arr)
words=tfidf.get_feature_names()
df_lda=pd.DataFrame(tfidf_words.toarray(),columns=words)
nmf = NMF(n_components=4, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)

print_top_words(nmf,words,4)

[['camera', 'model', 'money', 'smartphone'],
 ['performance', 'm30s', 'time', 'nice'],
 ['delivery', 'keybad', 'galaxy', 'ok'],
 ['battery', 'xcellent', 'quality', 'backup']]

## POS TAG NOUNS and ADJECTIVES

In [ ]:
data=pd.read_csv("samsungdata.csv")
data["review"]=data["review"].apply(lambda x:convert_emojis(x))
data["review"]=data["review"].apply(lambda x:x.lower()) 
data["review"]=data["review"].apply(lambda x:decontracted(x))
data["review"]=data["review"].apply(lambda x:removecharacters(x))
data["review"]=pd.DataFrame(data["review"].apply(lambda x:removestopwords(x)))

In [ ]:
def only_nouns_adjectives(review):
  lis= " ".join([each.strip(" ") for each,pos in nltk.pos_tag(review.split()) if (pos=="NN" or pos=="JJ")])
  return lis
data["title"]=data["title"].apply(lambda x:only_nouns_adjectives(x))


### LDA BOW

In [ ]:
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(data["review"])
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=10,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),4)

[['flipkart', 'best', 'value', 'worst'],
 ['super', 'awsome', 'quality', 'fine'],
 ['hours', 'display', 'battery', 'great'],
 ['best', 'camera', 'good', 'bad'],
 ['hangs', 'no', 'thanks', 'flipkart'],
 ['poor', 'excellent', 'good', 'box'],
 ['superb', 'samsung', 'buy', 'smartphone'],
 ['good', 'battery', 'camera', 'performance'],
 ['xcellent', 'awesome', 'money', 'worth'],
 ['nice', 'best', 'wesome', 'samsung']]

#### LDA again

In [ ]:
arr=np.array(print_top_words(lda,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=5,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),4)


[['fine', 'best', 'superb', 'hangs'],
 ['worth', 'battery', 'price', 'smartphone'],
 ['money', 'camera', 'good', 'poor'],
 ['hours', 'buy', 'xcellent', 'price'],
 ['great', 'box', 'samsung', 'flipkart']]

### LDA TFIDF

In [ ]:

tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(data["review"])
words=tfidf.get_feature_names()
df_bow=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=10,random_state=0)
lda.fit_transform(tfidf_words)
print_top_words(lda,words,5)

[['display', 'performance', 'best', 'battery', 'nicc'],
 ['super', 'awesome', 'flipkart', 'value', 'money'],
 ['thanks', 'suprrrrrrrrr', 'cell', 'tanks', 'perform'],
 ['bad', 'awsm', 'great', 'loving', 'prublam'],
 ['no', 'hangs', 'smooth', 'frequently', 'lovely'],
 ['exellent', 'sm', 'nice', 'isplay', 'life'],
 ['xcellent', 'superb', 'love', 'smartphone', 'flipkart'],
 ['good', 'camera', 'price', 'quality', 'nice'],
 ['backup', 'battery', 'camara', 'worth', 'loved'],
 ['nice', 'wesome', 'best', 'samsung', 'ok']]

#### LDA again

In [ ]:
arr=np.array(print_top_words(lda,words,5)).reshape(1,-1)[0]
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(arr)
words=tfidf.get_feature_names()
df_tfidf=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=4,random_state=0)
lda.fit_transform(df_tfidf)
print_top_words(lda,words,4)


[['flipkart', 'backup', 'tanks', 'loving'],
 ['suprrrrrrrrr', 'xcellent', 'sm', 'bad'],
 ['battery', 'worth', 'love', 'frequently'],
 ['nice', 'best', 'value', 'money']]

### NMF BOW

In [ ]:
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(data["review"])

nmf = NMF(n_components=10, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
bow_words=bow.get_feature_names()
print_top_words(nmf,bow_words,4)

[['good', 'battery', 'camera', 'display'],
 ['super', 'superb', 'great', 'display'],
 ['samsung', 'buy', 'm30s', 'loved'],
 ['nice', 'delivery', 'thanks', 'awsm'],
 ['quality', 'camera', 'hangs', 'using'],
 ['best', 'price', 'range', 'camera'],
 ['wesome', 'battery', 'money', 'xcellent'],
 ['performance', 'excellent', 'no', 'display'],
 ['use', 'ok', 'best', 'awsome'],
 ['flipkart', 'thank', 'love', 'worst']]

#### NMF again

In [ ]:
arr=np.array(print_top_words(nmf,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
nmf = NMF(n_components=5, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
bow_words=bow.get_feature_names()
print_top_words(nmf,bow_words,4)

[['display', 'hours', 'buy', 'worst'],
 ['battery', 'great', 'money', 'good'],
 ['camera', 'xcellent', 'online', 'backup'],
 ['best', 'quality', 'budget', 'hangs'],
 ['performance', 'love', 'samsung', 'better']]

### NMF TFIDF

In [ ]:
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(data["review"])
words=tfidf.get_feature_names()

In [ ]:
nmf = NMF(n_components=10, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)
print_top_words(nmf,words,6)

[['good', 'one', 'normal', 'choic', 'isplay', 'choice'],
 ['nice', 'awsm', 'phones', 'samsang', 'cell', 'osm'],
 ['super', 'quality', 'camera', 'also', 'performance', 'look'],
 ['wesome', 'outstanding', 'perfomance', 'loving', 'slow', 'gaming'],
 ['best', 'samsung', 'price', 'loved', 'range', 'buy'],
 ['battery', 'display', 'camera', 'backup', 'no', 'poor'],
 ['superb', 'expected', 'great', 'utstanding', 'got', 'galaxy'],
 ['xcellent', 'flipkart', 'smartphone', 'love', 'excellent', 'thanks'],
 ['money', 'awesome', 'value', 'much', 'delivery', 'worth'],
 ['bad', 'ok', 'thank', 'worst', 'hangs', 'working']]

#### NMF again -------------

In [ ]:
arr=np.array(print_top_words(nmf,words,5)).reshape(1,-1)[0]
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(arr)
words=tfidf.get_feature_names()
df_lda=pd.DataFrame(tfidf_words.toarray(),columns=words)
nmf = NMF(n_components=4, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)

print_top_words(nmf,words,4)

[['camera', 'excellent', 'awesome', 'thank'],
 ['love', 'no', 'perfomance', 'performance'],
 ['xcellent', 'range', 'awsm', 'bad'],
 ['smartphone', 'battery', 'normal', 'samsang']]

# Sentence Tokenisation

In [ ]:
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
review=pd.read_csv("samsungdata.csv")["review"].values

In [ ]:
import nltk.data

In [ ]:
sent=[]
for i in review:
  sent.extend(nltk.sent_tokenize(i))

In [ ]:
len(sent)

348

In [ ]:
df=pd.DataFrame(sent,columns=["review"])

In [ ]:
df.head()

,review
0,Bokeh mode is very excellent than any other ha...
1,Just awsome product the picture quality is goo...
2,I reccomend you to buy this product
3,Samsung galaxy m30s is most useful.But Samsung...
4,Where is i am disappointed with the selfie but...


In [ ]:
df.shape

(348, 1)

In [ ]:
df["review"]=df["review"].apply(lambda x:convert_emojis(x))
df["review"]=df["review"].apply(lambda x:x.lower()) 
df["review"]=df["review"].apply(lambda x:decontracted(x))
df["review"]=df["review"].apply(lambda x:removecharacters(x))
df["review"]=pd.DataFrame(df["review"].apply(lambda x:removestopwords(x)))

In [ ]:
#data["title"]=data["title"].apply(lambda x:only_nouns_adjectives(x))
#df["review"]=df["review"].apply(lambda x:only_nouns(x))

##sent tokanised 

### BOW LDA

In [ ]:
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(df["review"])
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=10,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),5)

[['wesome', 'flipkart', 'worst', 'ok', 'use'],
 ['super', 'samsung', 'quality', 'bad', 'camera'],
 ['xcellent', 'excellent', 'battery', 'thanks', 'camera'],
 ['good', 'camera', 'battery', 'super', 'one'],
 ['hangs', 'best', 'no', 'frequently', 'smooth'],
 ['good', 'poor', 'buy', 'online', 'though'],
 ['superb', 'good', 'like', 'buy', 'performance'],
 ['good', 'battery', 'camera', 'display', 'money'],
 ['battery', 'worth', 'good', 'backup', 'money'],
 ['nice', 'best', 'awesome', 'battery', 'price']]

#### LDA again

In [ ]:
arr=np.array(print_top_words(lda,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=5,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),4)


[['money', 'ok', 'performance', 'bad'],
 ['price', 'though', 'worth', 'smooth'],
 ['battery', 'good', 'hangs', 'samsung'],
 ['camera', 'buy', 'nice', 'flipkart'],
 ['best', 'super', 'wesome', 'online']]

### TFIDF LDA

In [ ]:
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(df["review"])
words=tfidf.get_feature_names()
df_bow=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=10,random_state=0)
lda.fit_transform(tfidf_words)
print_top_words(lda,words,5)

[['wesome', 'ok', 'smart', 'months', 'quality'],
 ['super', 'quality', 'love', 'flipkart', 'samsung'],
 ['excellent', 'great', 'loved', 'thank', 'camera'],
 ['xcellent', 'bad', 'battery', 'camera', 'one'],
 ['samsung', 'hangs', 'no', 'frequently', 'smooth'],
 ['recommend', 'worthless', 'exellent', 'sm', 'online'],
 ['superb', 'smartphone', 'sad', 'buy', 'good'],
 ['good', 'camera', 'battery', 'display', 'also'],
 ['worth', 'normal', 'money', 'wsm', 'camera'],
 ['nice', 'best', 'awesome', 'battery', 'price']]

#### (LDA AGAIN)

In [ ]:
arr=np.array(print_top_words(lda,words,5)).reshape(1,-1)[0]
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(arr)
words=tfidf.get_feature_names()
df_tfidf=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=4,random_state=0)
lda.fit_transform(df_tfidf)
print_top_words(lda,words,4)


[['hangs', 'bad', 'also', 'worth'],
 ['camera', 'battery', 'samsung', 'one'],
 ['quality', 'thank', 'sm', 'smartphone'],
 ['good', 'smooth', 'frequently', 'superb']]

### NMF TFIDF

In [249]:
nmf = NMF(n_components=10, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)
print_top_words(nmf,words,5)

[['camera', 'flipkart', 'xcellent', 'months', 'money'],
 ['battery', 'xcellent', 'flipkart', 'months', 'money'],
 ['good', 'xcellent', 'flipkart', 'months', 'money'],
 ['quality', 'bad', 'xcellent', 'flipkart', 'money'],
 ['samsung', 'xcellent', 'flipkart', 'money', 'loved'],
 ['no', 'one', 'awesome', 'loved', 'superb'],
 ['xcellent', 'buy', 'online', 'frequently', 'sad'],
 ['excellent', 'normal', 'best', 'great', 'smooth'],
 ['also', 'smartphone', 'love', 'price', 'recommend'],
 ['worthless', 'wsm', 'ok', 'super', 'hangs']]

### NMF BOW

In [250]:
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(data["review"])

nmf = NMF(n_components=10, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
bow_words=bow.get_feature_names()
print_top_words(nmf,bow_words,4)

[['good', 'battery', 'camera', 'display'],
 ['super', 'superb', 'great', 'display'],
 ['samsung', 'buy', 'm30s', 'loved'],
 ['nice', 'delivery', 'thanks', 'awsm'],
 ['quality', 'camera', 'hangs', 'using'],
 ['best', 'price', 'range', 'camera'],
 ['wesome', 'battery', 'money', 'xcellent'],
 ['performance', 'excellent', 'no', 'display'],
 ['use', 'ok', 'best', 'awsome'],
 ['flipkart', 'thank', 'love', 'worst']]

#### nmf again

In [251]:
arr=np.array(print_top_words(nmf,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
nmf = NMF(n_components=5, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
bow_words=bow.get_feature_names()
print_top_words(nmf,bow_words,4)

[['display', 'hours', 'buy', 'worst'],
 ['battery', 'great', 'money', 'good'],
 ['camera', 'xcellent', 'online', 'backup'],
 ['best', 'quality', 'budget', 'hangs'],
 ['performance', 'love', 'samsung', 'better']]

## POS TAGGING ONLY NOUNS

In [252]:
df["review"]=df["review"].apply(lambda x:only_nouns(x))

### BOW LDA

In [258]:
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(df["review"])
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=10,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),4)

[['price', 'love', 'range', 'samsung'],
 ['superb', 'voice', 'life', 'condition'],
 ['battery', 'samsung', 'camera', 'display'],
 ['battery', 'backup', 'ok', 'person'],
 ['quality', 'camera', 'nice', 'awesome'],
 ['smartphone', 'use', 'display', 'battery'],
 ['wesome', 'camera', 'super', 'performance'],
 ['money', 'm30s', 'delivery', 'budget'],
 ['buy', 'online', 'xcellent', 'delivery'],
 ['display', 'camera', 'battery', 'value']]

#### again LDA -------------------------------------------------------

In [261]:
arr=np.array(print_top_words(lda,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=5,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),4)


[['camera', 'display', 'person', 'm30s'],
 ['samsung', 'xcellent', 'buy', 'range'],
 ['quality', 'delivery', 'use', 'life'],
 ['battery', 'nice', 'superb', 'wid'],
 ['performance', 'network', 'super', 'handset']]

### BOW NMF  (results are better)

In [272]:
bow=CountVectorizer(ngram_range=(1,3))
bow_review=bow.fit_transform(df["review"])
nmf = NMF(n_components=10, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
print_top_words(nmf,bow.get_feature_names(),8)

[['battery',
  'camera',
  'display',
  'camera battery',
  'battery life',
  'life',
  'mah',
  'display battery'],
 ['performance',
  'budget',
  'handset',
  'mode',
  'cost',
  'handset budget',
  'camera display',
  'processer'],
 ['display',
  'super',
  'xcellent',
  'ok',
  'camera',
  'smartphone',
  'camera display',
  'credit super'],
 ['camera',
  'quality',
  'wesome',
  'camera quality',
  'quality battery',
  'look',
  'osm',
  'love'],
 ['backup',
  'superb',
  'battery backup',
  'battery',
  'experience',
  'awesome',
  'picture quality',
  'camara'],
 ['samsung',
  'buy',
  'flipkart',
  'online',
  'm30s',
  'love',
  'buy online',
  'purchase'],
 ['money', 'value', 'value money', 'delivery', 'time', 'mark', 'hand', 'box'],
 ['use',
  'nice',
  'play',
  'bit',
  'ok',
  'battery pack',
  'month',
  'super month bit'],
 ['price',
  'range',
  'front',
  'price range',
  'performance',
  'battery',
  'camera',
  'excellent'],
 ['power',
  'team',
  'power button',
  

#### again NMF---------------------

In [273]:
arr=np.array(print_top_words(nmf,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,2))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
nmf = NMF(n_components=4, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
bow_words=bow.get_feature_names()
print_top_words(nmf,bow_words,4)

[['battery', 'backup', 'quality', 'battery backup'],
 ['camera', 'quality', 'm30s', 'samsung'],
 ['price', 'range', 'money', 'value'],
 ['ok', 'performance', 'button', 'power']]

### TFIDF LDA

In [274]:
tfidf=TfidfVectorizer(ngram_range=(1,3))
tfidf_words=tfidf.fit_transform(df["review"])
words=tfidf.get_feature_names()
df_bow=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=10,random_state=0)
lda.fit_transform(tfidf_words)
print_top_words(lda,words,8)

[['love',
  'charge',
  'awesomespeechless',
  'cam',
  'voice',
  'battery good',
  'camera look',
  'good'],
 ['awesome',
  'range',
  'box',
  'nicc',
  'okay',
  'suprrrrrrrrr',
  'battery',
  'price range'],
 ['quality',
  'camera',
  'camera quality',
  'battery',
  'isplay',
  'thank',
  'smart',
  'purpose'],
 ['super', 'camera', 'samsung', 'nice', 'ok', 'date', 'function', 'worth'],
 ['xcellent',
  'smartphone',
  'perfomance',
  'excellent',
  'sm',
  'software',
  'anything',
  'waysome'],
 ['price',
  'battery',
  'mark',
  'wsm',
  'samsang',
  'till',
  'money good',
  'xcellent smartphone'],
 ['battery backup',
  'backup',
  'battery',
  'camera',
  'android',
  'perform',
  'worthless',
  'exellent'],
 ['money',
  'value',
  'value money',
  'time',
  'awsm',
  'flipkart',
  'light',
  'light camera'],
 ['samsung',
  'm30s',
  'sad',
  'mi',
  'samsung m30s',
  'samsung bro',
  'bro',
  'device'],
 ['wesome',
  'superb',
  'buy',
  'display',
  'buy online',
  'online',

#### Again LDF

In [279]:
arr=np.array(print_top_words(lda,words,5)).reshape(1,-1)[0]
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(arr)
words=tfidf.get_feature_names()
df_tfidf=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=4,random_state=0)
lda.fit_transform(df_tfidf)
print_top_words(lda,words,4)


[['android', 'display', 'm30s', 'awsm'],
 ['money', 'quality', 'backup', 'samsang'],
 ['perfomance', 'battery', 'love', 'samsang'],
 ['camera', 'time', 'charge', 'samsung']]

### TFIDF NMF

In [285]:
tfidf=TfidfVectorizer(ngram_range=(1,3))
tfidf_words=tfidf.fit_transform(data["review"])
words=tfidf.get_feature_names()

In [286]:
nmf = NMF(n_components=10, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)
print_top_words(nmf,words,6)

[['good', 'good samsung', 'good one', 'one', 'samsung', 'isplay good'],
 ['nice', 'phones', 'nice display', 'nice phones', 'nice awsm', 'osm nice'],
 ['wesome',
  'quality battery',
  'wesome camera',
  'battery',
  'wesome loving',
  'loving'],
 ['super', 'love', 'super nice', 'super camera', 'good love', 'super quality'],
 ['best', 'price', 'loved', 'good price', 'best price', 'samsung best'],
 ['superb',
  'great',
  'expected',
  'got',
  'got expected',
  'superb battery backup'],
 ['xcellent', 'money', 'flipkart', 'smartphone', 'thanks', 'excellent'],
 ['awesome', 'lovely', 'goods', 'thakarthu', 'exellent', 'worth money'],
 ['bad', 'ok', 'camera', 'camera quality', 'quality', 'worst'],
 ['battery', 'no', 'camera', 'battery backup', 'backup', 'display']]


#### again nmf

In [287]:
arr=np.array(print_top_words(nmf,words,6)).reshape(1,-1)[0]
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(arr)
words=tfidf.get_feature_names()
df_lda=pd.DataFrame(tfidf_words.toarray(),columns=words)
nmf = NMF(n_components=4, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)

print_top_words(nmf,words,4)

[['camera', 'super', 'quality', 'wesome'],
 ['battery', 'backup', 'money', 'superb'],
 ['good', 'price', 'samsung', 'best'],
 ['nice', 'display', 'phones', 'got']]